In [ ]:
!pip install -r '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/ocr_ancient_korean/recognition/requirements.txt'

In [1]:
import os
import argparse
import random
import time
from tqdm import tqdm
import yaml
import shutil
from psutil import virtual_memory
import multiprocessing
import numpy as np
import torch
from torch import nn, optim
from torchvision import transforms
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb

ImportError: ignored

In [ ]:
from checkpoint import (
    default_checkpoint,
    load_checkpoint,
    save_checkpoint,
    init_tensorboard,
    write_tensorboard,
    write_wandb
)

In [ ]:
from flags import Flags
from utils import set_seed, print_system_envs, get_optimizer, get_network, id_to_string
from utils import get_timestamp
from dataset import dataset_loader, START, PAD, load_vocab
from scheduler_ import CircularLRBeta, CustomCosineAnnealingWarmUpRestarts, TeacherForcingScheduler
from metrics_ import word_error_rate, sentence_acc, final_metric

In [ ]:
os.environ["WANDB_LOG_MODEL"] = "true"
os.environ["WANDB_WATCH"] = "all"